<a href="https://colab.research.google.com/github/CoolerTuro/IA_AssociationRule_DataSci/blob/main/65011212183_Lab8_datascience_168.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
basketData = pd.read_csv('/content/drive/My Drive/Data science/Market_Basket_Optimisation.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/datascience_lab/week7_associationrule/dataset/Market_Basket_Optimisation.csv'

In [ ]:
basketData.columns

In [ ]:
basketData.isnull().sum().sum()

In [ ]:
print(basketData.shape)

In [ ]:
import numpy as np
dataset = []
for i in range(0, basketData.shape[0]):
   temp = []
   for j in range(0, basketData.shape[1]):
       if not pd.isna(basketData.iat[i,j]):
           temp.append(basketData.iat[i,j])
   dataset.append(temp)
print(dataset)

In [ ]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)

In [ ]:
print(frequent_itemsets)

In [ ]:


from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np

# --- 0) ตั้งค่าไฟล์ + เกณฑ์ (แก้ได้) ---
CSV_PATH = "/content/drive/My Drive/Data science/Market_Basket_Optimisation.csv"
MIN_SUPPORT     = 0.003     # เดิมคุณใช้ 0.05; ลดเป็น 0.01 เพื่อให้มี itemset มากพอ
MIN_CONFIDENCE  = 0.15     # กำหนด threshold ที่เหมาะสม (เช่น 0.25)
MIN_LIFT        = 1.00     # >1 = เกิดร่วมกันเกินความบังเอิญ

# --- 1) โหลด CSV แบบตารางธุรกรรม (หลายคอลัมน์เป็น item text) ---
basketData = pd.read_csv(CSV_PATH)
print("shape =", basketData.shape)
_ = basketData.columns
_ = basketData.isnull().sum().sum()  # ตามโค้ดเดิมของคุณ
print("preview:\n", basketData.head(3))

# --- 2) แปลงเป็นรายการ (list of transactions) ตามโค้ดเดิมของคุณ ---
dataset = []
for i in range(0, basketData.shape[0]):
   temp = []
   for j in range(0, basketData.shape[1]):
       if not pd.isna(basketData.iat[i, j]):
           s = str(basketData.iat[i, j]).strip()
           if s != "" and s.lower() != "nan":
               temp.append(s)
   dataset.append(temp)

print("transactions example:", dataset[0][:10], "...")

# --- 3) One-hot ด้วย TransactionEncoder ---
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

# --- 4) หา frequent itemsets (Apriori) ---
from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(df, min_support=MIN_SUPPORT, use_colnames=True)
print("frequent_itemsets found:", len(frequent_itemsets))
print(frequent_itemsets.head())

# --- 5) สร้าง association rules + คำนวณ confidence / lift + กรองตาม threshold ---
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=MIN_CONFIDENCE)
rules = rules[rules["lift"] >= MIN_LIFT].copy()

# เติมความยาวของชุดหน้า/ชุดตาม เพื่อคัดกฎตามสเปค
rules["ante_len"] = rules["antecedents"].apply(len)
rules["cons_len"] = rules["consequents"].apply(len)

# เรียงกฎที่ "แรง" ตามลำดับ lift > confidence > support
rules_sorted = rules.sort_values(by=["lift", "confidence", "support"], ascending=False)

print("\n=== Top 10 rules (sorted by lift, confidence, support) ===")
print(rules_sorted[["antecedents","consequents","support","confidence","lift"]].head(10))

# --- 6) เลือก 3 กฎตามโจทย์ ---
def pick_rule(df, a_len, b_len):
    sub = df[(df["ante_len"]==a_len) & (df["cons_len"]==b_len)]
    if sub.empty:
        return None
    return sub.iloc[0]  # df ถูก sort แล้ว

rule1 = pick_rule(rules_sorted, 1, 1)  # 1-itemset -> 1-itemset
rule2 = pick_rule(rules_sorted, 2, 1)  # 2-itemset -> 1-itemset
rule3 = pick_rule(rules_sorted, 2, 2)  # 2-itemset -> 2-itemset

selected = [r for r in [rule1, rule2, rule3] if r is not None]

# --- 7) พิมพ์ผลกฎที่คัดเลือกเพื่อใช้เขียนรายงาน ---
def as_list(fs):
    # frozenset -> string
    return ", ".join(list(fs)) if hasattr(fs, "__iter__") else str(fs)

print("\n=== Selected 3 rules for explanation ===")
if not selected:
    print("ไม่พบกฎที่ผ่านเกณฑ์ ลองลด MIN_SUPPORT / MIN_CONFIDENCE หรือ MIN_LIFT")
else:
    for i, r in enumerate(selected, 1):
        A = as_list(r["antecedents"])
        B = as_list(r["consequents"])
        print(f"Rule {i}: {A} -> {B}")
        print(f"  Support    = {r['support']:.4f} ({r['support']:.2%})")
        print(f"  Confidence = {r['confidence']:.4f} ({r['confidence']:.2%})")
        print(f"  Lift       = {r['lift']:.4f}")
        print("-"*60)

# หมายเหตุ: กฎแบบ (2-itemset)->(2-itemset) มักหาได้ยากในชุดข้อมูลจริง
if rule3 is None:
    print("\nหมายเหตุ: ไม่พบกฎ (2-itemset)->(2-itemset) ที่ผ่านเกณฑ์ในชุดข้อมูลนี้ "
          "เพราะต้องเกิดสินค้า 4 รายการพร้อมกันในธุรกรรมเดียว")


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
shape = (7500, 20)
preview:
     shrimp    almonds avocado vegetables mix green grapes whole weat flour  \
0  burgers  meatballs    eggs            NaN          NaN              NaN   
1  chutney        NaN     NaN            NaN          NaN              NaN   
2   turkey    avocado     NaN            NaN          NaN              NaN   

  yams cottage cheese energy drink tomato juice low fat yogurt green tea  \
0  NaN            NaN          NaN          NaN            NaN       NaN   
1  NaN            NaN          NaN          NaN            NaN       NaN   
2  NaN            NaN          NaN          NaN            NaN       NaN   

  honey salad mineral water salmon antioxydant juice frozen smoothie spinach  \
0   NaN   NaN           NaN    NaN               NaN             NaN     NaN   
1   NaN   NaN           NaN    NaN               NaN            

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

transactions example: ['burgers', 'meatballs', 'eggs'] ...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

frequent_itemsets found: 1442
    support             itemsets
0  0.020267            (almonds)
1  0.008800  (antioxydant juice)
2  0.004800          (asparagus)
3  0.033200            (avocado)
4  0.004533        (babies food)

=== Top 10 rules (sorted by lift, confidence, support) ===
                                   antecedents  \
1816                 (soup, frozen vegetables)   
1810            (olive oil, frozen vegetables)   
1613        (whole wheat pasta, mineral water)   
1815                              (milk, soup)   
1504                            (tomato sauce)   
1813  (milk, frozen vegetables, mineral water)   
1809                         (milk, olive oil)   
282                            (fromage blanc)   
1798          (frozen vegetables, ground beef)   
1502                 (tomato sauce, spaghetti)   

                             consequents   support  confidence      lift  
1816               (milk, mineral water)  0.003067    0.383333  7.986111  
1810       

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag